<a href="https://colab.research.google.com/github/Swastik200/30DaysOfDSA/blob/main/Input_Attn_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp "/content/drive/MyDrive/CMAPSS.zip" "/content"
!unzip CMAPSS.zip

Archive:  CMAPSS.zip
  inflating: CMAPSS/readme.txt       
  inflating: CMAPSS/RUL_FD001.txt    
  inflating: CMAPSS/RUL_FD002.txt    
  inflating: CMAPSS/RUL_FD003.txt    
  inflating: CMAPSS/RUL_FD004.txt    
  inflating: CMAPSS/test_FD001.txt   
  inflating: CMAPSS/test_FD002.txt   
  inflating: CMAPSS/test_FD003.txt   
  inflating: CMAPSS/test_FD004.txt   
  inflating: CMAPSS/train_FD001.txt  
  inflating: CMAPSS/train_FD002.txt  
  inflating: CMAPSS/train_FD003.txt  
  inflating: CMAPSS/train_FD004.txt  
  inflating: CMAPSS/x.txt            


In [4]:
path='/content/CMAPSS/'

In [7]:
train_df = pd.read_csv(path+'train_FD001.txt', delimiter=' ', header=None)
columns = ['unit_number', 'time_in_cycles', 'operational_setting_1', 'operational_setting_2', 'operational_setting_3'] + [f'sensor_{i}' for i in range(1, 24)]
train_df.columns = columns




In [9]:
test_df = pd.read_csv(path+'test_FD001.txt', delimiter=' ', header=None)
columns = ['unit_number', 'time_in_cycles', 'operational_setting_1', 'operational_setting_2', 'operational_setting_3'] + [f'sensor_{i}' for i in range(1, 24)]
test_df.columns = columns


In [10]:
max_cycles = train_df.groupby('unit_number')['time_in_cycles'].max().reset_index()
max_cycles.columns = ['unit_number', 'max_cycles']

In [42]:
display(train_df)

,unit_number,time_in_cycles,operational_setting_1,operational_setting_2,operational_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,NaN,NaN,30
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,NaN,NaN,29
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,NaN,NaN,28
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,NaN,NaN,27
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,NaN,NaN,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,8.4956,0.03,397,2388,100.0,38.49,22.9735,NaN,NaN,2
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,8.5139,0.03,395,2388,100.0,38.30,23.1594,NaN,NaN,1
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,8.5646,0.03,398,2388,100.0,38.44,22.9333,NaN,NaN,0
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,8.5389,0.03,395,2388,100.0,38.29,23.0640,NaN,NaN,-1


In [11]:
max_cycles = test_df.groupby('unit_number')['time_in_cycles'].max().reset_index()
max_cycles.columns = ['unit_number', 'max_cycles']

In [12]:
train_df = train_df.merge(max_cycles, on='unit_number', how='left')
train_df['RUL'] = train_df['max_cycles'] - train_df['time_in_cycles']
train_df.drop('max_cycles', axis=1, inplace=True)


In [13]:
test_df = test_df.merge(max_cycles, on='unit_number', how='left')
test_df['RUL'] = test_df['max_cycles'] - test_df['time_in_cycles']
test_df.drop('max_cycles', axis=1, inplace=True)


In [14]:
def prepare_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data)-1:
            break
        seq_x, seq_y = data[i:end_ix, :-1], data[end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


In [15]:
n_steps = 20

In [16]:
data = train_df.drop(columns=['unit_number', 'time_in_cycles']).values


In [17]:
data = test_df.drop(columns=['unit_number', 'time_in_cycles']).values


In [77]:
X, y = prepare_sequences(data, n_steps)

In [82]:
from keras.models import Model
from keras.layers import LSTM, Dense, Attention, Input

# Define Input
inputs = Input(shape=(n_steps, X.shape[2]))

# Define LSTM layer
lstm_output = LSTM(100, activation='relu', return_sequences=True)(inputs)

# Define query and value tensors
query = lstm_output  # Use LSTM output as query
value = lstm_output  # Use LSTM output as value

# Apply Attention mechanism
attention_output = Attention()([query, value])

# Apply Dense layer
output = Dense(1)(attention_output)

# Create the model
model = Model(inputs=inputs, outputs=output)

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=100, verbose=0)


In [83]:
train_pred = model.predict(X)

409/409 [==============================] - 4s 8ms/step


In [84]:
test_pred = model.predict(X)

409/409 [==============================] - 3s 7ms/step


In [88]:
# Import necessary modules
import numpy as np
from sklearn.metrics import mean_squared_error

# Investigate the shapes of y and train_pred_aggregated
print(f"Shape of y: {y.shape}")
print(f"Shape of train_pred_aggregated: {train_pred_aggregated.shape}")

# Fix the issue by reshaping y to match the shape of train_pred_aggregated
y = y.reshape(train_pred_aggregated.shape)

# Calculate RMSE for train set
train_rmse = sqrt(mean_squared_error(y, train_pred_aggregated))
print(f"Train RMSE: {train_rmse}")

Shape of y: (13076, 1)
Shape of train_pred_aggregated: (261520, 1)


ValueError: cannot reshape array of size 13076 into shape (261520,1)

In [89]:
import numpy as np

# Print the shapes of y and train_pred_aggregated
print(f"Shape of y: {y.shape}")
print(f"Shape of train_pred_aggregated: {train_pred_aggregated.shape}")

Shape of y: (13076, 1)
Shape of train_pred_aggregated: (261520, 1)


In [91]:
print(f"Shape of y: {y.shape}")
print(f"Shape of train_pred_aggregated: {train_pred_aggregated.shape}")

Shape of y: (13076, 1)
Shape of train_pred_aggregated: (261520, 1)


In [92]:
if y.shape[0] != train_pred_aggregated.shape[0]:
    print("Number of rows in y does not match the number of rows in train_pred_aggregated.")

Number of rows in y does not match the number of rows in train_pred_aggregated.


In [93]:
if y.shape[0] != train_pred_aggregated.shape[0]:
    y = np.concatenate((y, np.full((train_pred_aggregated.shape[0] - y.shape[0], 1), np.nan)))

In [94]:
y = y.reshape(train_pred_aggregated.shape)

In [95]:
print(f"Shape of y after reshape: {y.shape}")
print(f"Shape of train_pred_aggregated: {train_pred_aggregated.shape}")

Shape of y after reshape: (261520, 1)
Shape of train_pred_aggregated: (261520, 1)


In [97]:
# Check for NaN values in the train_pred array
np.isnan(train_pred).sum()

261520

In [101]:
train_pred[np.isnan(train_pred)] = 0

In [103]:
print(f"y: {y.shape}")
print(f"train_pred: {train_pred.shape}")

y: (261520, 1)
train_pred: (261520, 1)


In [106]:
print(f"Number of NaN values in y: {np.isnan(y).sum()}")
print(f"Number of NaN values in train_pred: {np.isnan(train_pred).sum()}")

Number of NaN values in y: 0
Number of NaN values in train_pred: 0


In [108]:
print(y.shape)

(13076,)


In [109]:
print(y.shape, train_pred.shape)

(13076,) (261520,)


In [116]:
train_pred_reshaped = train_pred.reshape(-1, 1)


In [121]:
from math import sqrt
from sklearn.metrics import mean_squared_error

# Reshape train_pred to have the same number of samples as y
train_pred_reshaped = train_pred[:y.shape[0]]

# Calculate RMSE for train set
train_rmse = sqrt(mean_squared_error(y, train_pred_reshaped))
print(f"Train RMSE: {train_rmse}")


Train RMSE: 92.6201661735329


In [132]:
from sklearn.metrics import mean_absolute_error, r2_score

In [133]:
print("y:", y)
print("train_pred:", train_pred)

y: [10.  9.  8. ...  2.  1.  0.]
train_pred: [0. 0. 0. ... 0. 0. 0.]


In [136]:
from sklearn.metrics import mean_absolute_error, r2_score

# Reshape train_pred to have the same number of samples as y
train_pred_reshaped = train_pred.reshape(-1, 1)[:y.shape[0]]

# Calculate MAE for train set
train_mae = mean_absolute_error(y, train_pred_reshaped)

# Calculate R2 for train set
train_r2 = r2_score(y, train_pred_reshaped)

print(f"Train R2: {train_r2}, MAE: {train_mae}")


Train R2: -2.0479926934232546, MAE: 75.92115325787702


In [105]:
y = y[~np.isnan(y)]
train_pred = train_pred[~np.isnan(train_pred)]

In [40]:
from keras.models import Sequential

In [44]:
model.compile(optimizer='adam', loss='mse')

In [45]:
model.fit(X, y, epochs=100, verbose=0)

In [46]:
train_pred = model.predict(X)

409/409 [==============================] - 4s 9ms/step


In [47]:
test_pred = model.predict(X)

409/409 [==============================] - 3s 8ms/step


In [62]:
import math

In [63]:
from sklearn.metrics import mean_squared_error

In [72]:
print(y.shape)


(13076, 1)


In [73]:
print(y)


[[10.]
 [ 9.]
 [ 8.]
 ...
 [ 2.]
 [ 1.]
 [ 0.]]


In [74]:
print(train_pred.shape)
print(train_pred)

(13076, 0)
[]


In [56]:
!pip install scikit-learn

In [36]:
attention_output = Dot(axes=[1, 1])([attention_weights, Permute([2, 1])(value)])

In [32]:
!pip install keras

In [33]:
from tensorflow.keras.layers import Dot